In [1]:
from numpy.core.fromnumeric import amin
import requests
from bs4 import BeautifulSoup
from datetime import date
import re 
import numpy as np 
import pandas as pd 
from IPython.display import clear_output
from time import sleep
from itertools import islice
import time

# Funciones

In [2]:
def get_index(iterable, value, n):
    matches = (idx for idx, val in enumerate(iterable) if value in val)
    return next(islice(matches, n-1, n), None)

In [3]:
def clean_data(df):
    df['titulo'] = df['titulo'].str.replace(r"\n|\t","")
    df['titulo'] = df['titulo'].str.replace(r"\n|\t","")
    df['region'] = df['region'].str.replace(r"\n|\t","")
    df['tipo inmueble'] = df['tipo inmueble'].str.replace(r"\n|\t","")
    df['proposito'] = df['proposito'].str.replace(r"\n|\t","")
    df['direccion'] = df['direccion'].str.replace("/",",")
    mask = pd.notna(df['precio'])
    df.loc[mask, 'precio'] = df.loc[mask, 'precio'].str.replace(".", "")
    df.loc[mask, 'precio'] = df.loc[mask, 'precio'].astype(int)
    mask1 = pd.notna(df['gastos comunes'])
    df.loc[mask1, 'gastos comunes'] = df.loc[mask1, 'gastos comunes'].str.replace("CLP","")
    mask5 = pd.notna(df['estacionamiento'])
    df.loc[mask5, 'estacionamiento'] = df.loc[mask5, 'estacionamiento'].astype(int)
    mask6 = pd.notna(df['bodega'])
    df.loc[mask6, 'bodega'] = df.loc[mask6, 'bodega'].astype(int)
    mask7 = pd.notna(df['ambientes'])
    df.loc[mask7, 'ambientes'] = df.loc[mask7, 'ambientes'].astype(int)
    
    df['habitaciones'] = df['habitaciones'].str.replace(r"-\s+\d\s\w*","").str.replace('Dormitorio',"").str.strip()
    df_habitaciones = df['habitaciones'].tolist()
    habitaciones = []
    for i in df_habitaciones:
        if len(str(i).split('-')) != 1:
            habitaciones.append(i.split('-')[-1].strip(' '))
        else:
            habitaciones.append(i)
    df['habitaciones'] = habitaciones
    df['habitaciones'] = df['habitaciones'].astype(float)
    
    df['banos'] = df['banos'].str.replace(r"-\s+\d\s\w*","").str.replace(r'Baño|s',"").str.strip()
    df_banos = df['banos'].tolist()
    banos = []
    for i in df_banos:
        if len(str(i).split('-')) != 1:
            banos.append(i.split('-')[-1].strip(' '))
        else:
            banos.append(i)
    df['banos'] = banos
    df['banos'] = df['banos'].astype(float)
    
    mask2 = pd.notna(df['m2_terraza'])
    df.loc[mask2, 'm2_terraza'] = df.loc[mask2, 'm2_terraza'].str.replace(r"-.*$|m.","")
    df.loc[mask2, 'm2_terraza'] = df.loc[mask2, 'm2_terraza'].astype(float)
    mask3 = pd.notna(df['m2_utiles'])
    df.loc[mask3, 'm2_utiles'] = df.loc[mask3, 'm2_utiles'].str.replace(r"-.*$|m.","")
    df.loc[mask3, 'm2_utiles'] = df.loc[mask3, 'm2_utiles'].astype(float)
    mask4 = pd.notna(df['m2 totales'])
    df.loc[mask4, 'm2 totales'] = df.loc[mask4, 'm2 totales'].str.replace(r"-.*$|m.","")
    df.loc[mask4, 'm2 totales'] = df.loc[mask4, 'm2 totales'].astype(float)
    df['precio por m2'] = df['precio']/df['m2 totales']
    

    return df

In [4]:
def arriendos(region,pages,type,scope):  

    urls = []
    titles = []
    address = []
    prices = []
    total_m2 = []
    util_m2 = []
    terraza = []
    terraza_m2 = []
    ambientes = []
    rooms = []
    toilets = []
    publication_date = []
    parking = []
    common_spends = []
    olds = []
    garage = []
    orientation = []
    floors_numbers = []
    apartment_floor_number = []
    apartments_per_floor = []
    tipo_inmueble = []
    proposito = []
    region1 = []

    print("Web Scraping Portal Inmobiliario")

    print(f'Buscando {type} para {scope}....')
    
    #Iterar por pagina para encontrar las urls de cada oferta de depto y almacenar los resultados en una lista llamada urls
    for i in range(1,pages*50,50):
        main_url = 'https://www.portalinmobiliario.com/'+scope.lower().replace(" ","-")+'/'+type+'/'+region+'/_Desde_'+ str(i)
        main_response = requests.get(main_url)
        main_soup = BeautifulSoup(main_response.text,'html.parser')
        containers = main_soup.find_all('li',{'class':'ui-search-layout__item'})

        for container in containers: 
            urls.append(container.find('a',class_='ui-search-result__content-wrapper ui-search-link')['href'])

    counter = 0 
    for url in urls:
        try:
            for i in range(0,20):
                response = requests.get(url, allow_redirects=True)
                soup = BeautifulSoup(response.text, 'html.parser')

                time.sleep(2)

                head1 = []
                attr = soup.find_all('tr',class_='andes-table__row')
                for a in attr:
                    th = a.find('th',class_='andes-table__header andes-table__header--left ui-pdp-specs__table__column ui-pdp-specs__table__column-title')
                    if th != None:
                        head1.append(th.text)

                if ('Superficie total' in head1 \
                or 'Superficie útil' in head1) \
                and len(soup.find('h1', class_='ui-pdp-title')) != 0 \
                and soup.find('div', class_='ui-pdp-media ui-vip-location__subtitle ui-pdp-color--BLACK') != None:
                    break

            counter +=1
        except:
            while True:
                response = requests.get(url, allow_redirects=True)
                soup = BeautifulSoup(response.text, 'html.parser')

                time.sleep(2)

                if soup.find_all('tbody', class_='andes-table__body') != None \
                and len(soup.find('h1', class_='ui-pdp-title')) != 0 \
                and soup.find('div', class_='ui-pdp-media ui-vip-location__subtitle ui-pdp-color--BLACK') != None:
                    break

            counter +=1                


        # Región

        region1.append(region)

        # Departamento/casa

        tipo_inmueble.append(type)

        # Proposito

        proposito.append(scope)

        # Título

        try:
            titles.append(soup.find('h1',class_='ui-pdp-title').text)
        except (AttributeError,IndexError):
            titles.append(np.nan)
        
        # Dirección

        try:
            add = soup.find('div',class_='ui-pdp-media ui-vip-location__subtitle ui-pdp-color--BLACK').text
            address1 = add.split('Ver')[0]
            address.append(address1)
        except (AttributeError,IndexError):
            address.append(np.nan)

        # Precios

        try:
            price = soup.find_all('span',class_='andes-money-amount__fraction')
            if len(price) > 1:
                prices.append(price[1].text)
            else:
                prices.append(price[0].text)
        except (AttributeError,IndexError):
            prices.append(np.nan)

        # Características

        try:
            attribute = soup.find_all('tr',class_='andes-table__row')

            for a in attribute:
                heading = a.find('th',class_='andes-table__header andes-table__header--left ui-pdp-specs__table__column ui-pdp-specs__table__column-title').text
                values = a.find('td',class_='andes-table__column andes-table__column--left ui-pdp-specs__table__column').text

                if heading == 'Dormitorios':
                    rooms.append(values)

                if heading == 'Baños':
                    toilets.append(values)

                if heading == 'Ambientes':
                    ambientes.append(values)

                if heading == 'Superficie total':
                    total_m2.append(values)

                if heading == 'Superficie útil':
                    util_m2.append(values)

                if heading == 'Superficie de terraza':
                    terraza_m2.append(values)

                if heading == 'Superficie de terraza':
                  if values != None:
                    terraza.append('Si')
                  else:
                    terraza.append('No')

                if heading =='Estacionamientos':
                    parking.append(values)

                if heading == 'Gastos comunes':
                    common_spends.append(values)

                if heading == 'Bodegas':
                    garage.append(values)

                if heading == 'Antigüedad':
                    olds.append(values)

                if heading == 'Orientación':
                    orientation.append(values)

                if heading == 'Número de piso de la unidad':
                    apartment_floor_number.append(values)

                if heading == 'Departamentos por piso':
                    apartments_per_floor.append(values)

                if heading == 'Cantidad de pisos':
                    floors_numbers.append(values)
        except (AttributeError,IndexError):

            rooms.append(np.nan)
            toilets.append(np.nan)
            ambientes.append(np.nan)
            total_m2.append(np.nan)
            util_m2.append(np.nan)
            terraza_m2.append(np.nan)
            terraza.append(np.nan)
            parking.append(np.nan)
            garage.append(np.nan)
            olds.append(np.nan)
            common_spends.append(np.nan)
            orientation.append(np.nan)
            apartment_floor_number.append(np.nan)
            apartments_per_floor.append(np.nan)
            floors_numbers.append(np.nan)


        #Clausula para mantener cantidad de variables por cada oferta de departamento, en caso de no existir esa variable se llena con nan
        if len(titles) != counter:
            titles.append(np.nan)
            
        if len(address) != counter:
            address.append(np.nan)
            
        if len(publication_date) != counter:
            publication_date.append(np.nan)
            
        if len(prices) != counter:
            prices.append(np.nan)
            
        if len(rooms) != counter:
            rooms.append(np.nan)
            
        if len(toilets) != counter:
            toilets.append(np.nan)
            
        if len(util_m2) != counter:
            util_m2.append(np.nan)
            
        if len(total_m2) != counter:
            total_m2.append(np.nan)
            
        if len(terraza) != counter:
            terraza.append(np.nan)
            
        if len(terraza_m2) != counter:
            terraza_m2.append(np.nan)
            
        if len(parking) != counter:
            parking.append(np.nan)
            
        if len(garage) != counter:
            garage.append(np.nan)
            
        if len(olds) != counter:
            olds.append(np.nan)
            
        if len(common_spends) != counter:
            common_spends.append(np.nan)
            
        if len(orientation) != counter:
            orientation.append(np.nan)
            
        if len(apartment_floor_number) != counter:
            apartment_floor_number.append(np.nan)
            
        if len(apartments_per_floor) != counter:
            apartments_per_floor.append(np.nan)
            
        if len(floors_numbers) != counter:
            floors_numbers.append(np.nan)
            
        if len(ambientes) != counter:
            ambientes.append(np.nan)


        print(f"{type} en {scope} encontrados: {len(titles)}")
        clear_output(wait=True)

    print(f"Total informacion extraida de {type}s en {scope}: {len(titles)}")
    print('Web Scraping Completado!\n')



    df = pd.DataFrame({ 'titulo':titles,
                        'region':region,
                        'tipo inmueble':tipo_inmueble,
                        'proposito':proposito,
                        'direccion':address,
                        'orientacion':orientation,
                        'antiguedad':olds,
                        'm2 totales':total_m2,
                        'm2_utiles':util_m2,
                        'terraza':terraza,
                        'm2_terraza':terraza_m2,
                        'ambientes':ambientes,
                        'habitaciones':rooms,
                        'banos':toilets,
                        'estacionamiento':parking,
                        'bodega':garage,
                        'piso de apartamento':apartment_floor_number,
                        'cantidad de pisos edificio':floors_numbers,
                        'apartamentos por piso':apartments_per_floor,
                        'gastos comunes': common_spends,
                        'precio':prices,
                        'url':urls})

    return df

In [5]:
def ventas(region,pages,type,scope):  

    urls = []
    titles = []
    address = []
    prices = []
    total_m2 = []
    util_m2 = []
    terraza = []
    terraza_m2 = []
    ambientes = []
    rooms = []
    toilets = []
    publication_date = []
    parking = []
    common_spends = []
    olds = []
    garage = []
    orientation = []
    floors_numbers = []
    apartment_floor_number = []
    apartments_per_floor = []
    tipo_inmueble = []
    proposito = []
    region1 = []

    print("Web Scraping Portal Inmobiliario")

    print(f'Buscando {type} para {scope}....')
    
    #Iterar por pagina para encontrar las urls de cada oferta de depto y almacenar los resultados en una lista llamada urls
    for i in range(1,pages*50,50):
        main_url = 'https://www.portalinmobiliario.com/'+scope.lower().replace(" ","-")+'/'+type+'/'+region+'/_Desde_'+ str(i)
        main_response = requests.get(main_url)
        main_soup = BeautifulSoup(main_response.text,'html.parser')
        containers = main_soup.find_all('li',{'class':'ui-search-layout__item'})

        for container in containers: 
            urls.append(container.find('a',class_='ui-search-result__content-wrapper ui-search-link')['href'])

    counter = 0 
    for url in urls:
        try:
            for i in range(0,20):
                response = requests.get(url, allow_redirects=True)
                soup = BeautifulSoup(response.text, 'html.parser')

                time.sleep(2)

                head1 = []
                attr = soup.find_all('tr',class_='andes-table__row')
                for a in attr:
                    th = a.find('th',class_='andes-table__header andes-table__header--left ui-pdp-specs__table__column ui-pdp-specs__table__column-title')
                    if th != None:
                        head1.append(th.text)

                if ('Superficie total' in head1 \
                or 'Superficie útil' in head1) \
                and len(soup.find('h1', class_='ui-pdp-title')) != 0 \
                and soup.find('div', class_='ui-pdp-media ui-vip-location__subtitle ui-pdp-color--BLACK') != None:
                    break

            counter +=1
        except:
            while True:
                response = requests.get(url, allow_redirects=True)
                soup = BeautifulSoup(response.text, 'html.parser')

                time.sleep(2)

                if soup.find_all('tbody', class_='andes-table__body') != None \
                and len(soup.find('h1', class_='ui-pdp-title')) != 0 \
                and soup.find('div', class_='ui-pdp-media ui-vip-location__subtitle ui-pdp-color--BLACK') != None:
                    break

            counter +=1 

        # Región

        region1.append(region)

        # Departamento/casa

        tipo_inmueble.append(type)

        # Proposito

        proposito.append(scope)

        # Título

        try:
            titles.append(soup.find('h1',class_='ui-pdp-title').text)
        except (AttributeError,IndexError):
            titles.append(np.nan)
        
        # Dirección

        try:
            add = soup.find('div',class_='ui-pdp-media ui-vip-location__subtitle ui-pdp-color--BLACK').text
            address1 = add.split('Ver')[0]
            address.append(address1)
        except (AttributeError,IndexError):
            address.append(np.nan)

        # Precios

        try:
            price = soup.find_all('span',class_='andes-money-amount__fraction')
            if len(price) > 1:
                prices.append(price[1].text)
            else:
                prices.append(price[0].text)
        except (AttributeError,IndexError):
            prices.append(np.nan)

        # Características

        try:
            attribute = soup.find_all('tr',class_='andes-table__row')

            for a in attribute:
                heading = a.find('th',class_='andes-table__header andes-table__header--left ui-pdp-specs__table__column ui-pdp-specs__table__column-title').text
                values = a.find('td',class_='andes-table__column andes-table__column--left ui-pdp-specs__table__column').text

                if heading == 'Dormitorios':
                    rooms.append(values)

                if heading == 'Baños':
                    toilets.append(values)

                if heading == 'Ambientes':
                    ambientes.append(values)

                if heading == 'Superficie total':
                    total_m2.append(values)

                if heading == 'Superficie útil':
                    util_m2.append(values)

                if heading == 'Superficie de terraza':
                    terraza_m2.append(values)

                if heading == 'Superficie de terraza':
                  if values != None:
                    terraza.append('Si')
                  else:
                    terraza.append('No')

                if heading =='Estacionamientos':
                    parking.append(values)

                if heading == 'Gastos comunes':
                    common_spends.append(values)

                if heading == 'Bodegas':
                    garage.append(values)

                if heading == 'Antigüedad':
                    olds.append(values)

                if heading == 'Orientación':
                    orientation.append(values)

                if heading == 'Número de piso de la unidad':
                    apartment_floor_number.append(values)

                if heading == 'Departamentos por piso':
                    apartments_per_floor.append(values)

                if heading == 'Cantidad de pisos':
                    floors_numbers.append(values)
        except (AttributeError,IndexError):

            rooms.append(np.nan)
            toilets.append(np.nan)
            ambientes.append(np.nan)
            total_m2.append(np.nan)
            util_m2.append(np.nan)
            terraza_m2.append(np.nan)
            terraza.append(np.nan)
            parking.append(np.nan)
            garage.append(np.nan)
            olds.append(np.nan)
            common_spends.append(np.nan)
            orientation.append(np.nan)
            apartment_floor_number.append(np.nan)
            apartments_per_floor.append(np.nan)
            floors_numbers.append(np.nan)


        #Clausula para mantener cantidad de variables por cada oferta de departamento, en caso de no existir esa variable se llena con nan
        if len(titles) != counter:
            titles.append(np.nan)
            
        if len(address) != counter:
            address.append(np.nan)
            
        if len(publication_date) != counter:
            publication_date.append(np.nan)
            
        if len(prices) != counter:
            prices.append(np.nan)
            
        if len(rooms) != counter:
            rooms.append(np.nan)
            
        if len(toilets) != counter:
            toilets.append(np.nan)
            
        if len(util_m2) != counter:
            util_m2.append(np.nan)
            
        if len(total_m2) != counter:
            total_m2.append(np.nan)
            
        if len(terraza) != counter:
            terraza.append(np.nan)
            
        if len(terraza_m2) != counter:
            terraza_m2.append(np.nan)
            
        if len(parking) != counter:
            parking.append(np.nan)
            
        if len(garage) != counter:
            garage.append(np.nan)
            
        if len(olds) != counter:
            olds.append(np.nan)
            
        if len(common_spends) != counter:
            common_spends.append(np.nan)
            
        if len(orientation) != counter:
            orientation.append(np.nan)
            
        if len(apartment_floor_number) != counter:
            apartment_floor_number.append(np.nan)
            
        if len(apartments_per_floor) != counter:
            apartments_per_floor.append(np.nan)
            
        if len(floors_numbers) != counter:
            floors_numbers.append(np.nan)
            
        if len(ambientes) != counter:
            ambientes.append(np.nan)


        print(f"{type} en {scope} encontrados: {len(titles)}")
        clear_output(wait=True)

    print(f"Total informacion extraida de {type}s en {scope}: {len(titles)}")
    print('Web Scraping Completado!\n')



    df = pd.DataFrame({ 'titulo':titles,
                        'region':region,
                        'tipo inmueble':tipo_inmueble,
                        'proposito':proposito,
                        'direccion':address,
                        'orientacion':orientation,
                        'antiguedad':olds,
                        'm2 totales':total_m2,
                        'm2_utiles':util_m2,
                        'terraza':terraza,
                        'm2_terraza':terraza_m2,
                        'ambientes':ambientes,
                        'habitaciones':rooms,
                        'banos':toilets,
                        'estacionamiento':parking,
                        'bodega':garage,
                        'piso de apartamento':apartment_floor_number,
                        'cantidad de pisos edificio':floors_numbers,
                        'apartamentos por piso':apartments_per_floor,
                        'gastos comunes': common_spends,
                        'precio':prices,
                        'url':urls})

    return df

In [6]:
def WebScrapping_PI(region, pages, type, scope):
    if scope == "venta":
        return ventas(region, pages, type, scope)
    elif scope == "arriendo":
        return arriendos(region, pages, type, scope)
    else:
        print("Error: type debe ser 'venta' o 'arriendo'")

# Ejecuciones

In [7]:
df = WebScrapping_PI(region="santiago-metropolitana",pages=2,type='departamento',scope='arriendo')

Total informacion extraida de departamentos en arriendo: 100
Web Scraping Completado!



In [8]:
df1 = WebScrapping_PI(region="coquimbo",pages=2,type='departamento',scope='arriendo')

Total informacion extraida de departamentos en arriendo: 100
Web Scraping Completado!



In [9]:
df2 = WebScrapping_PI(region="santiago-metropolitana",pages=3,type='casa',scope='arriendo')

Total informacion extraida de casas en arriendo: 43
Web Scraping Completado!



In [10]:
df3 = WebScrapping_PI(region="coquimbo",pages=3,type='casa',scope='venta')

Total informacion extraida de casas en venta: 150
Web Scraping Completado!



# Concatenación de dataframes y limpieza de datos

In [11]:
"""
En df_final, dentro de la función verá que tiene los distintos df creados en el paso anterior.
Para utilizarlo, debe adaptar la parte que esta dentro de los corchetes de pd.concat([...]) según la cantidad
de df que haya creado.

Es decir, si solamente ha creado df y df1, entonces:
df_final = pd.concat([df, df1])

Si ha creado, df, df1, df2, df3, df4, df5:
df_final = pd.concat([df, df1, df2, df3, df4, df5])

"""
df_final = pd.concat([df, df1, df2, df3])
df_final = clean_data(df_final)
df_final = df_final.fillna(0)
df_final

C:\Users\ACERNI~1\AppData\Local\Temp/ipykernel_15012/4142505433.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['titulo'] = df['titulo'].str.replace(r"\n|\t","")
C:\Users\ACERNI~1\AppData\Local\Temp/ipykernel_15012/4142505433.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['titulo'] = df['titulo'].str.replace(r"\n|\t","")
C:\Users\ACERNI~1\AppData\Local\Temp/ipykernel_15012/4142505433.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['region'] = df['region'].str.replace(r"\n|\t","")
C:\Users\ACERNI~1\AppData\Local\Temp/ipykernel_15012/4142505433.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tipo inmueble'] = df['tipo inmueble'].str.replace(r"\n|\t","")
C:\Users\ACERNI~1\AppData\Local\Temp/ipykernel_15012/4142505433.py:6: FutureWarning: The default value of rege

,titulo,region,tipo inmueble,proposito,direccion,orientacion,antiguedad,m2 totales,m2_utiles,terraza,...,banos,estacionamiento,bodega,piso de apartamento,cantidad de pisos edificio,apartamentos por piso,gastos comunes,precio,url,precio por m2
0,Edificio Tarapaca- Vista Poniente Piso 15,santiago-metropolitana,departamento,arriendo,"Tarapaca 890, Bulnes, Santiago, RM (Metropolit...",Poniente,0,32.78,28.78,Si,...,1.0,0,0,15,27,17,0,310000,https://www.portalinmobiliario.com/MLC-1524654...,9456.985967
1,Edificio José Miguel Carrera- Vista Oriente Pi...,santiago-metropolitana,departamento,arriendo,"José Miguel Carrera 321, Ejército - Toesca, Sa...",Oriente,0,42.00,42.00,0,...,1.0,1,1,12,14,8,0,400000,https://www.portalinmobiliario.com/MLC-1424661...,9523.809524
2,Edificio Carmen- Vista Sur Piso 4,santiago-metropolitana,departamento,arriendo,"Carmen 121, Santa Isabel, Santiago, RM (Metrop...",Sur,0,34.00,32.00,Si,...,1.0,0,1,4,9,22,0,360000,https://www.portalinmobiliario.com/MLC-1372473...,10588.235294
3,Edificio Matucana- Vista Poniente Piso 7,santiago-metropolitana,departamento,arriendo,"Matucana 31, Barrio Yungay, Santiago, RM (Metr...",Poniente,0,39.42,36.25,Si,...,1.0,0,0,7,10,15,0,325000,https://www.portalinmobiliario.com/MLC-1355105...,8244.545916
4,Edificio Matucana- Vista Oriente Piso 8,santiago-metropolitana,departamento,arriendo,0,0,0,0.00,0.00,0,...,0.0,0,0,0,0,0,0,426000,https://www.portalinmobiliario.com/MLC-1443304...,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Amplia Casa Remodelada En Peñuelas,coquimbo,casa,venta,"Juan Budinich, Coquimbo, Chile, Peñuelas, Coqu...",0,0 años,234.00,183.00,0,...,3.0,1,1,0,2,0,0,220000000,https://www.portalinmobiliario.com/MLC-1371692...,940170.940171
146,"Preciosa, Amplia, Luminosa Y Cómoda Casa Rosar...",coquimbo,casa,venta,"Del Atardecer 4200 - 4500, Coquimbo, Chile, Pe...",NP,30 años,585.00,200.00,0,...,3.0,4,2,0,1,0,0,378092083,https://www.portalinmobiliario.com/MLC-1372647...,646311.252991
147,Preciosa Casa Loteo San Marino Peñuelas Vista ...,coquimbo,casa,venta,"Río Lauca 3254, Coquimbo, 1802100 - 1802400, C...",0,0,0.00,0.00,0,...,0.0,0,0,0,0,0,0,245759854,https://www.portalinmobiliario.com/MLC-1372610...,0.000000
148,Amplia Y Bonita Casa En Colina Del Pino,coquimbo,casa,venta,"Ambrosio Mansilla, La Serena, Chile, San Joaqu...",0,0 años,380.00,182.00,0,...,3.0,2,0,0,1,0,0,253250358,https://www.portalinmobiliario.com/MLC-1524764...,666448.310526


# Obtención de latitud y longitud

In [12]:
direcciones = df_final['direccion'].tolist()
latitud = []
longitud = []


for i in direcciones:
    # URL de la API de MapQuest
    url = f'http://www.mapquestapi.com/geocoding/v1/address?key=90MNhFPFBSjxV2THAjkQFEt7pjkvZXWF&location={i}'

    # Obtener la respuesta de la API
    response = requests.get(url).json()

    # Obtener la latitud y longitud
    latitud1 = response['results'][0]['locations'][0]['latLng']['lat']
    longitud1 = response['results'][0]['locations'][0]['latLng']['lng']

    latitud.append(latitud1)
    longitud.append(longitud1)

#Almacenar resultados en Data Frame
df_final['latitud'] = latitud
df_final['longitud'] = longitud
df_final

# Reordenar las columnas
df_final = df_final[['titulo', 'region', 'tipo inmueble', 'proposito', 'direccion',
       'orientacion', 'antiguedad', 'm2 totales', 'm2_utiles', 'terraza',
       'm2_terraza', 'ambientes', 'habitaciones', 'banos', 'estacionamiento',
       'bodega', 'piso de apartamento', 'cantidad de pisos edificio',
       'apartamentos por piso', 'gastos comunes', 'precio', 'precio por m2', 'latitud', 'longitud', 'url']]


# Exportar Output a Excel

In [13]:
# Exportar file
"""
Para exportar de forma correcta debería seguir la siguiente sintaxis:
df_final.to_excel('path/NombreArchivo.xlsx')

Por ejemplo:
df_final.to_excel('C:/Users/Usuario1/Documents/Output.xlsx')

Es importante utilizar el slash /, ya que es normal que windows copie y pegue el slash invertido \, el cual
no es reconocido por python para los paths.

"""
df_final.to_excel('C:/Users/Acer Nitro 5/Documents/Fundacion/Output.xlsx')